# Evaluation of Experimental Results

## Prepare Data

Provide paths to the directory with experimental results and cleaned preprocessed data for the analysis

In [ ]:
predictions_dir = '../output/predictions/'
data_real_path = '../data/electricity_cleaned_preprocessed.csv'

In [ ]:
import seaborn as sns

MAE = 'MAE'
MAPE = 'MAPE'
SMAPE = 'sMAPE'
MSE = 'MSE'
R2 = 'R2'
MASE = 'MASE'

ARIMA = 'ARIMA'
PROPHET = 'Prophet'
SVR = 'SVR'
RF = 'RF'
LSTM = 'LSTM'
TRANSFORMER = 'Transformer'

OFFICE = 'Office'
EDUCATION = 'Education'
LODGING = 'Lodging/resedential'
PUBLIC = 'Public services'
ASSEMBLY = 'Entertainment/public assembly'
INDUSTRIAL = 'Manufacturing/industrial'
PARKING = 'Parking'
SERVICES = 'Services'
FOOD = 'Food sales and service'
OTHER = 'Other'


metric_columns = [MAE, SMAPE, MAPE, MSE, R2, MASE]

model_name_mapper = {
    'arima': ARIMA,
    'prophet': PROPHET,
    'svr': SVR,
    'rf': RF,
    'lstm': LSTM,
    'transformer': TRANSFORMER
}

type_name_mapper = {
    'office': OFFICE,
    'education': EDUCATION,
    'lodging': LODGING,
    'public': PUBLIC,
    'assembly': ASSEMBLY,
    'industrial': INDUSTRIAL,
    'parking': PARKING,
    'services': SERVICES,
    'food': FOOD,
    'other': OTHER
}

name_mapper = {**model_name_mapper, **type_name_mapper}

# model_order = [AR, ARIMA, ARIMA_NOCOVARS,ARIMA_NOSEASON, PROPHET, SVR, RF, LSTM, TRANSFORMER]
MODEL_ORDER = [ARIMA, PROPHET, SVR, RF, LSTM, TRANSFORMER]
TYPE_ORDER = [OFFICE, EDUCATION, LODGING, PUBLIC, ASSEMBLY, INDUSTRIAL, PARKING, SERVICES, FOOD, OTHER]

# Based on colorblind
COLOR_PALETTE = []
for color in [0,1,2,3,4,7]:
    COLOR_PALETTE.append(sns.color_palette("colorblind")[color])
COLOR_PALETTE

def filter_relevant_models(df):
    return df[df['model'].isin(MODEL_ORDER)]

In [ ]:
import pandas as pd
import os

data_predictions = pd.read_csv(os.path.join(predictions_dir, 'predictions.csv'), parse_dates=True, index_col=['timestamp', 'model', 'training_length'])
data_real = pd.read_csv(data_real_path, parse_dates=True, index_col='timestamp')
data_real = data_real.loc[data_real.index, data_real.columns.isin(data_predictions.columns)]
data_times = pd.read_csv(os.path.join(predictions_dir, 'times.csv'))
data_times = data_times.replace(name_mapper)

data_predictions.columns.name = 'building'
data_predictions = data_predictions.stack(future_stack=True)
data_predictions.name = 'predicted'
data_predictions = data_predictions.reset_index()

data_real['model'] = 'real'
data_real.columns.name = 'building'
data_real = data_real.stack(future_stack=True)
data_real.name = 'real'
data_real = data_real.reset_index()

data = data_predictions.merge(data_real, on=['timestamp', 'building'])
data = data.replace(name_mapper)
data

In [ ]:
# Create naive forecaster
data['naive'] = data.groupby(['model', 'building', 'training_length'])['real'].shift(1)

## Performance Metrics

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score

def calculate_increase(df, metric_column_name):
    return df.apply(lambda x: x[metric_column_name]/df[df['model'] == x['model']].loc[14, metric_column_name], axis=1)

def smape(y_true, y_pred) -> float: 
    return np.mean(np.abs(y_pred - y_true) / ((np.abs(y_pred) + np.abs(y_true))/2))*100

def mase(y_true, y_pred, naive) -> float: 
    return np.mean(np.abs(y_pred - y_true) / np.mean(np.abs(naive-y_true)))

def apply_metric(dataframe, metric_function):
    if metric_function == mase:
        return metric_function(y_true=dataframe['real'], y_pred=dataframe['predicted'], naive=dataframe['naive'])
    return metric_function(y_true=dataframe['real'], y_pred=dataframe['predicted'])

def calculate_metrics(dataframe):
    metrics = pd.DataFrame()
    metrics[MAE] = dataframe.groupby(['model', 'building', 'training_length'])[['real', 'predicted']].apply(lambda x:apply_metric(x, mean_absolute_error))
    metrics[MAPE] = dataframe.groupby(['model', 'building', 'training_length'])[['real', 'predicted']].apply(lambda x: apply_metric(x, mean_absolute_percentage_error))
    metrics[SMAPE] = dataframe.groupby(['model', 'building', 'training_length'])[['real', 'predicted']].apply(lambda x: apply_metric(x, smape))
    metrics[MSE] = dataframe.groupby(['model', 'building', 'training_length'])[['real', 'predicted']].apply(lambda x: apply_metric(x, mean_squared_error))
    metrics[R2] = dataframe.groupby(['model', 'building', 'training_length'])[['real', 'predicted']].apply(lambda x: apply_metric(x, r2_score))
    metrics[MASE] = dataframe.groupby(['model', 'building', 'training_length'])[['real', 'predicted', 'naive']].apply(lambda x: apply_metric(x, mase))
    return metrics

metrics = calculate_metrics(data)
metrics

In [ ]:
# Aggregated by model

by_model = metrics.groupby(['model', 'training_length'])[metric_columns].agg('mean').reset_index().set_index('training_length')
by_model['mase_percentage'] = calculate_increase(by_model, metric_column_name=MASE) * 100
by_model

In [ ]:
by_model.groupby(['training_length']).agg({'MASE': 'min'})

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt

sns.set_style("whitegrid")

def metrics_visualization(data, metric_column, aggregation_function='mean', **kwargs):
    agg_metric_by_model = data.groupby(['model', 'training_length'])[metric_columns].agg(aggregation_function).reset_index().set_index('training_length')
    ax = sns.lineplot(data=agg_metric_by_model, x='training_length', y=metric_column, hue='model', hue_order=MODEL_ORDER)
    ax.legend(title=None)
    ax.set_xticks(agg_metric_by_model.index.unique())
    ax.set_xlabel('Training data size (days)')
    return ax

def metrics_visualization_bar(data, metric_column, aggregation_function='mean', **kwargs):
    agg_metric_by_model = data.groupby(['model', 'training_length'])[metric_columns].agg(aggregation_function).reset_index().set_index('training_length')
    ax = sns.barplot(data=agg_metric_by_model, x='training_length', y=metric_column, hue='model', hue_order=MODEL_ORDER)
    ax.legend(title=None)
    ax.set_xlabel('Training data size (days)')
    return ax

def metrics_visualization_box(data, metric_column, palette=None, **kwargs):
    palette = palette if palette else COLOR_PALETTE
    agg_metric_by_model = data.reset_index().set_index('training_length')
    ax = sns.boxplot(data=agg_metric_by_model, x='training_length', y=metric_column, hue='model', showfliers=False,
                      hue_order=MODEL_ORDER, palette=palette, showmeans=True,
                        meanprops={'marker':'o','markerfacecolor':'white','markeredgecolor':'black'})
    ax.legend(title=None)
    ax.set_xlabel('Training data size (days)')
    sns.despine(left=True)
    return ax

def metrics_visualization_box_alt(data, metric_column, palette=None, type_order=None, width=None, **kwargs):
    palette = palette if palette else 'Set3'
    type_order = type_order if type_order else TYPE_ORDER
    width = width if width else 0.8
    agg_metric_by_model = data.reset_index().set_index('training_length')      
    ax = sns.boxplot(data=agg_metric_by_model, x='training_length', y=metric_column, hue='type', showfliers=False,
                      hue_order=type_order, palette=palette, width=width, showmeans=True,
                        meanprops={'marker':'o','markerfacecolor':'white','markeredgecolor':'black', 'markersize':5})
    ax.legend(title=None)
    ax.set_xlabel('Training data size (days)')
    sns.despine(left=True)
    return ax

def heatmap_visualization(data, metric_column, aggregation_function='mean', annot=False, **kwargs):
    day_metrics_agg = data.groupby(['training_length', 'day'])[metric_column].agg(aggregation_function).reset_index()
    day_metrics_agg = day_metrics_agg.pivot(index='training_length', columns='day', values=metric_column)
    ax = sns.heatmap(data=day_metrics_agg, cmap="flare", annot=annot, cbar=False)
    ax.set_xlabel('Day in forecasting horizon')
    ax.set_ylabel('Training data size (days)')
    return ax

In [ ]:
plt.rcParams.update({'font.size': 12})
ax = metrics_visualization_box(metrics, MASE)
ax.set_ylabel('MASE')
plt.tight_layout()

ax

In [ ]:
metrics_detailed = metrics.reset_index()
metrics_detailed[['location', 'type', 'building']] = metrics_detailed['building'].str.split('_', expand=True)
metrics_detailed = metrics_detailed.set_index('training_length')
metrics_detailed = metrics_detailed.replace(name_mapper)
metrics_detailed

In [ ]:
g = sns.FacetGrid(metrics_detailed, col='location', aspect=2)
g.map_dataframe(metrics_visualization_box, metric_column=MASE)
g.add_legend()

In [ ]:
plt.rcParams.update({'font.size': 12})
g = sns.FacetGrid(metrics_detailed, col='model', aspect=2, sharey=False, col_wrap=2, col_order=MODEL_ORDER)
g.map_dataframe(metrics_visualization_box_alt, metric_column=MASE, type_outliers=['Services'])
g.add_legend()
sns.move_legend(g, 'lower center',  bbox_to_anchor=(0.4, -0.08), ncol=5)

In [ ]:
metrics_adjusted = metrics_detailed[metrics_detailed.type.isin(['Services'])]
plt.rcParams.update({'font.size': 12})
g = sns.FacetGrid(metrics_adjusted, col='model', aspect=0.7, sharey=False, col_wrap=3, col_order=MODEL_ORDER).tight_layout()
g.map_dataframe(metrics_visualization_box_alt, metric_column=MASE, hue_order=TYPE_ORDER[-1], type_order=['Services'], palette=[COLOR_PALETTE[-1]])
g.add_legend()
sns.move_legend(g, 'lower center',  bbox_to_anchor=(0.5, -0.04))

In [ ]:
metrics_adjusted = metrics_detailed[~metrics_detailed.type.isin(['Services'])]
plt.rcParams.update({'font.size': 12})
g = sns.FacetGrid(metrics_adjusted, col='model', aspect=2, sharey=False, col_wrap=3, col_order=MODEL_ORDER).tight_layout()
g.map_dataframe(metrics_visualization_box_alt, metric_column=MASE, type_outliers=['Services'], hue_order=[SERVICES])
g.add_legend()
sns.move_legend(g, 'lower center',  bbox_to_anchor=(0.4, -0.08), ncol=5)

In [ ]:
first_days = [pd.Timestamp(day).date() for day in ('2017-04-03', '2017-08-07', '2017-10-02', '2017-12-11')]

data['day'] = data.apply(lambda row: min([abs((row.name.date() - first_date).days) for first_date in first_days]) + 1, axis=1)
day_metrics = data.groupby('day').apply(calculate_metrics, include_groups=False).reset_index()
day_metrics

In [ ]:
plt.rcParams.update({'font.size': 12})
g = sns.FacetGrid(day_metrics, col='model', aspect=1.1, col_wrap=2, col_order=MODEL_ORDER, height=2.8).tight_layout()
g.map_dataframe(heatmap_visualization, metric_column=MASE, annot=True)

## Execution Time Evaluation

In [ ]:
def times_visualization(df):
    ax = sns.lineplot(data=df, x='training_length', y='process_time_h', hue='model', hue_order=MODEL_ORDER, palette=COLOR_PALETTE)
    ax.legend(title=None, frameon=False)
    sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
    ax.set_xticks(df.index.unique())
    ax.set_xlabel('Training data size (days)')
    ax.set_ylabel('CPU execution time (hours)')
    sns.despine(bottom=True)
    return ax

def times_visualization_bar(df, log_scale=True):
    ax = sns.barplot(data=df, x='process_time_h', y='training_length', hue='model', hue_order=MODEL_ORDER, palette=COLOR_PALETTE, orient='y')
    for i, container in enumerate(ax.containers):
        datapoints = container.datavalues
        percentages = np.round(datapoints/datapoints[0]*100)
        labels = [f'{int(number)} %' for number in percentages]
        ax.bar_label(container, labels=labels, color=COLOR_PALETTE[i] , fontsize=10, padding=5)
    ax.legend(title=None, frameon=False)
    sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
    ax.set_ylabel('Training data size (days)')
    ax.set_xlabel('CPU execution time (hours)')
    if log_scale:
        ax.set_xscale("log")
    sns.despine(bottom=True)
    return ax


def times_increase_visualization_bar(df, log_scale=True):
    ax = sns.barplot(data=df, y='training_length', x='process_time_increase_percentage', hue='model', hue_order=MODEL_ORDER, palette=COLOR_PALETTE, orient='y')
    ax.legend(title=None, frameon=True)
    # sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
    ax.set_ylabel('Training data size (days)')
    ax.set_xlabel('CPU execution time increase (percentage)')
    if log_scale:
        ax.set_xscale("log")
    sns.despine(bottom=True)
    return ax

In [ ]:
sum_times_by_model = data_times.groupby(['model', 'training_length']).sum().reset_index().set_index('training_length')
sum_times_by_model[['process_time_h', 'thread_time_h', 'wall_time_h']] =  sum_times_by_model[['process_time_ns', 'thread_time_ns', 'wall_time_ns']] / (3600 * 10**9)
sum_times_by_model

In [ ]:
sum_times_by_model['process_time_increase_percentage'] = calculate_increase(sum_times_by_model, metric_column_name='process_time_h') * 100
sum_times_by_model['thread_time_increase_percentage'] = calculate_increase(sum_times_by_model, metric_column_name='thread_time_h') * 100
sum_times_by_model['wall_time_increase_percentage'] = calculate_increase(sum_times_by_model, metric_column_name='wall_time_h') * 100
sum_times_by_model

In [ ]:
plt.rcParams.update({'font.size': 12})
ax = times_visualization_bar(sum_times_by_model, log_scale=False)

In [ ]:
plt.rcParams.update({'font.size': 12})
sns.set_style("whitegrid")
times_visualization_bar(sum_times_by_model, log_scale=True)
plt.tight_layout()

In [ ]:
plt.rcParams.update({'font.size': 12})
ax = times_increase_visualization_bar(sum_times_by_model, log_scale=False)
plt.tight_layout()

## Combined Performance Metrics and Execution Time Evaluation

In [ ]:
import seaborn as sns

def combined_visualization(df, performance_metric, time_metric='process_time_h'):
    ax = sns.scatterplot(x=time_metric, y=performance_metric, data=df, hue='model', hue_order=MODEL_ORDER, palette=COLOR_PALETTE)
    return ax

In [ ]:
mean_metric_by_model = metrics.groupby(['model', 'training_length'])[metric_columns].agg('mean').reset_index().set_index('training_length')
combined_by_model = sum_times_by_model.reset_index().merge(mean_metric_by_model.reset_index(), on=['training_length', 'model']).set_index('training_length')
combined_by_model = filter_relevant_models(combined_by_model)

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset

plt.rcParams.update({'font.size': 12})

ax = combined_visualization(combined_by_model, time_metric='process_time_h', performance_metric=MASE)
ax.set_ylabel('MASE')
ax.set_xlabel('CPU execution time (hours)')
ax.legend(title=None, frameon=False)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

# Create inset axes
axins = inset_axes(ax, width="20%", height="40%", loc=1)

# Plot zoomed data in inset
sns.scatterplot(ax=axins, x='process_time_h', y=MASE, data=combined_by_model, hue='model', hue_order=MODEL_ORDER, palette=COLOR_PALETTE)

# Set limits for zoomed region
x1, x2, y1, y2 = -0.2, 0.5, 2, 3  # Adjust these values as needed
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)
axins.set_xlabel(None)
axins.set_ylabel(None)
axins.get_legend().remove()

# Add zoom indicator
mark_inset(ax, axins, loc1=2, loc2=4, fc="none", ec="0.5")


ax
plt.tight_layout()
# plt.savefig("results_time_mase.pdf")
plt.savefig("results_time_mase.png", dpi=360)

In [ ]:
ax = combined_visualization(combined_by_model, time_metric='process_time_h', performance_metric=MASE)
ax.set_ylabel('MASE')
ax.set_xlabel('CPU execution time (hours)')
ax

In [ ]:
ax = combined_visualization(combined_by_model, time_metric='process_time_increase_percentage', performance_metric=MASE)
ax.set_ylabel('MASE')
ax.set_xlabel('CPU execution time increase (percentage)')
ax